In [14]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re
import json
import pickle

In [15]:
page = requests.get('https://www.newtral.es/zona-verificacion/fakes/')

In [16]:
#print ("pagina", page)
print ("pagina status code", page.status_code)
#print ("contenido de la pagina", page.content)

pagina status code 200


In [17]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup)
#print(soup.prettify())

In [18]:
def get_urls(noticias):
    
    page = requests.get('https://www.newtral.es/wp-json/wp/v2/posts?per_page='+ str(noticias) +'&offset=0&categories=59&_locale=user')

    for e in page.json():
        enlace = e['link']
        lista_enlaces.append(enlace)
        
    return lista_enlaces

In [39]:
def scraping_article(url):
    
    #article = {"Title":"Titulo_noticia", "Publication_date":"Fecha_noticia", "Content":"Article_content"}

    page = requests.get(url)

    soup3 = BeautifulSoup(page.content, 'html.parser')

    title = soup3.find("h1",class_="entry-title c-article__title").get_text()
    #print(title)
    
    date = soup3.find("time",class_="entry-date c-article__date published updated").get_text().strip('\n')
    #print(date)
    
    tags = soup3.find("ul", class_="tags-links c-pills uk-visible@m")
    tags2 = tags.find_all("a", class_="c-pills__item")
    
    tag_list = []
    
    for t in tags2:
        tag_list.append(t.get_text())
    
    content_list = []
    body = soup3.find("div", class_="entry-content")
    body2 = body.find_all("p", class_="")
    
    for e in body2:
        content_list.append(e.get_text().replace("\xa0", " "))
    
    content = ''.join(content_list[:-1])
    
    article = {"Title": title, "Tags": tag_list, "Publication_date": date, "Content": content, "URL": url, "Fuente": "Newtral.es"}
    
    return article


In [40]:
noticias = 1 #En esta web tenemos 13 noticias de portada y luego se van sumando de 12 en 12 (Podemos seleccionar un número intermedio)
lista_enlaces = []

lista_enlaces = get_urls(noticias)

#print(len(lista_enlaces))
lista_enlaces

lista_articulos = []   
for e in lista_enlaces:
    articulo = scraping_article(e)
    lista_articulos.append(articulo)


In [41]:
print(len(lista_enlaces))
print(lista_enlaces)

1
['https://www.newtral.es/la-foto-de-la-estatua-de-lorca-en-granada-con-una-esvastica-en-la-frente-ni-es-actual-ni-se-hizo-en-una-de-las-manifestaciones-contra-el-gobierno/20200529/']


In [47]:
print(lista_articulos)
#print(lista_articulos[0]['Tags'][0])

[{'Title': 'La foto de la estatua de Federico García Lorca en Granada con una esvástica en la frente ni es actual ni se hizo en una de las manifestaciones contra el Gobierno', 'Tags': ['coronavirus', 'estado de alarma', 'fakes', 'Federico García Lorca', 'Granada', 'manifestación'], 'Publication_date': '29 mayo 2020', 'Content': 'Nos habéis preguntado a través de nuestro servicio de verificación por varios tuits que están compartiendo una foto del monumento a Federico García Lorca situado en la avenida de la Constitución de Granada con una esvástica dibujada en la frente. En los tuits se acusa a los manifestantes que el sábado pasado se congregaron en la localidad, además de en varias ciudades de todo el territorio, convocados por VOX y que pedían la dimisión de Pedro Sánchez. Según la delegación de Gobierno en Granada se reunieron cerca de 1.000 coches y 150 motos.Estos son algunos de los tuits, “Federico García Lorca, tras el paso de los ‘demócratas’ en Granada”, “Cuando los fascistas

In [23]:
from datetime import date

date = date.today().strftime("%d-%m-%Y")

filename = 'Newtral_FN_' + date
print(filename)

outfile = open(filename,'wb')

pickle.dump(lista_articulos, outfile)
outfile.close()

Newtral_FN_30-05-2020


In [24]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()


print(new_dict==lista_articulos)
print(type(new_dict))
new_dict

True
<class 'list'>


[{'Title': 'La foto de la estatua de Federico García Lorca en Granada con una esvástica en la frente ni es actual ni se hizo en una de las manifestaciones contra el Gobierno',
  'Publication_date': '29 mayo 2020',
  'Content': 'Nos habéis preguntado a través de nuestro servicio de verificación por varios tuits que están compartiendo una foto del monumento a Federico García Lorca situado en la avenida de la Constitución de Granada con una esvástica dibujada en la frente. En los tuits se acusa a los manifestantes que el sábado pasado se congregaron en la localidad, además de en varias ciudades de todo el territorio, convocados por VOX y que pedían la dimisión de Pedro Sánchez. Según la delegación de Gobierno en Granada se reunieron cerca de 1.000 coches y 150 motos.Estos son algunos de los tuits, “Federico García Lorca, tras el paso de los ‘demócratas’ en Granada”, “Cuando los fascistas se manifiestan es para hacer cosas como esta”, incluso también encontramos menciones a VOX, “Los mal n